# Retrieval Augmented Generation (RAG) and Vector Databases

In [1]:
!pip install getenv openai==1.12.0

In [3]:
import os
import pandas as pd
import numpy as np
import openai

## Creating our Knowledge base

Creating a Azure Cosmos DB database


In [1]:
pip install azure-cosmos

Note: you may need to restart the kernel to use updated packages.


In [4]:
## create your cosmoss db on Azure CLI using the following commands
## az login
## az group create -n <resource-group-name> -l <location>
## az cosmosdb create -n <cosmos-db-name> -r <resource-group-name>
## az cosmosdb list-keys -n <cosmos-db-name> -g <resource-group-name>

## Once done navigate to data explorer and create a new database and a new container


In [62]:
from azure.cosmos import CosmosClient

# Initialize Cosmos Client
url = os.getenv('COSMOS_DB_ENDPOINT')
key = os.getenv('COSMOS_KEY')
print(url,key[:10])
clientCosmos = CosmosClient(url, credential=key)

# Select database
database_name = 'classdata'
database = clientCosmos.get_database_client(database_name)

# Select container
container_name = 'embeddings'
container = database.get_container_client(container_name)



https://damcosmos.documents.azure.com:443/ p3vG5uYg7G


In [20]:
import pandas as pd

# Initialize an empty DataFrame
df = pd.DataFrame(columns=['path', 'text'])

print(os.getcwd())
current_directory = os.getcwd()
data_folder = os.path.join(current_directory, 'data')
folder_contents = os.listdir(data_folder)
print(f"Contents of 'data' folder: {folder_contents}")



# splitting our data into chunks
data_paths= ["data/frameworks.md?WT.mc_id=academic-105485-koreyst", "data/own_framework.md?WT.mc_id=academic-105485-koreyst", "data/perceptron.md?WT.mc_id=academic-105485-koreyst"]
data_paths= ["data/frameworks.md", "data/own_framework.md", "data/perceptron.md"]

#for path in data_paths:
#    with open(path, 'r', encoding='utf-8') as file:
#        file_content = file.read()
#
#    # Append the file path and text to the DataFrame
#    df = df.append({'path': path, 'text': file_content}, ignore_index=True)
#
#

# List to hold DataFrame rows
rows = []

for path in data_paths:
    with open(path, 'r', encoding='utf-8') as file:
        file_content = file.read()
     # Append the file path and text to the list of rows
    rows.append({'path': path, 'text': file_content})

# Convert the list of rows to a DataFrame
df = pd.concat([pd.DataFrame([row]) for row in rows], ignore_index=True)

# Display the DataFrame
df.head

/workspaces/generative-ai-for-beginners/15-rag-and-vector-databases
Contents of 'data' folder: ['own_framework.md', 'perceptron.md', 'frameworks.md']


<bound method NDFrame.head of                     path                                               text
0     data/frameworks.md  # Neural Network Frameworks\n\nAs we have lear...
1  data/own_framework.md  # Introduction to Neural Networks. Multi-Layer...
2     data/perceptron.md  # Introduction to Neural Networks: Perceptron\...>

In [29]:
def split_text(text, max_length, min_length):
    words = text.split()
    chunks = []
    current_chunk = []

    for word in words:
        current_chunk.append(word)
        if len(' '.join(current_chunk)) < max_length and len(' '.join(current_chunk)) > min_length:
            chunks.append(' '.join(current_chunk))
            current_chunk = []

    # If the last chunk didn't reach the minimum length, add it anyway
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks
pd.set_option('display.max_rows', 5)
pd.set_option('display.max_columns', 5)
pd.set_option('display.max_colwidth', 300)
# Assuming analyzed_df is a pandas DataFrame and 'output_content' is a column in that DataFrame
splitted_df = df.copy()
splitted_df['chunks'] = splitted_df['text'].apply(lambda x: split_text(x, 400, 300))

splitted_df

,path,text,chunks
0,data/frameworks.md,"# Neural Network Frameworks\n\nAs we have learned already, to be able to train neural networks efficiently we need to do two things:\n\n* To operate on tensors, eg. to multiply, add, and compute some functions such as sigmoid or softmax\n* To compute gradients of all expressions, in order to per...","[# Neural Network Frameworks As we have learned already, to be able to train neural networks efficiently we need to do two things: * To operate on tensors, eg. to multiply, add, and compute some functions such as sigmoid or softmax * To compute gradients of all expressions, in order to perform g..."
1,data/own_framework.md,"# Introduction to Neural Networks. Multi-Layered Perceptron\n\nIn the previous section, you learned about the simplest neural network model - one-layered perceptron, a linear two-class classification model.\n\nIn this section we will extend this model into a more flexible framework, allowing us ...","[# Introduction to Neural Networks. Multi-Layered Perceptron In the previous section, you learned about the simplest neural network model - one-layered perceptron, a linear two-class classification model. In this section we will extend this model into a more flexible framework, allowing us to: *..."
2,data/perceptron.md,"# Introduction to Neural Networks: Perceptron\n\nOne of the first attempts to implement something similar to a modern neural network was done by Frank Rosenblatt from Cornell Aeronautical Laboratory in 1957. It was a hardware implementation called ""Mark-1"", designed to recognize primitive geomet...","[# Introduction to Neural Networks: Perceptron One of the first attempts to implement something similar to a modern neural network was done by Frank Rosenblatt from Cornell Aeronautical Laboratory in 1957. It was a hardware implementation called ""Mark-1"", designed to recognize primitive geometri..."


In [35]:
# Assuming 'chunks' is a column of lists in the DataFrame splitted_df, we will split the chunks into different rows
flattened_df = splitted_df.explode('chunks')
print(len(splitted_df))
print(len(flattened_df))
flattened_df.head()


3
57


,path,text,chunks
0,data/frameworks.md,"# Neural Network Frameworks\n\nAs we have learned already, to be able to train neural networks efficiently we need to do two things:\n\n* To operate on tensors, eg. to multiply, add, and compute some functions such as sigmoid or softmax\n* To compute gradients of all expressions, in order to per...","# Neural Network Frameworks As we have learned already, to be able to train neural networks efficiently we need to do two things: * To operate on tensors, eg. to multiply, add, and compute some functions such as sigmoid or softmax * To compute gradients of all expressions, in order to perform gr..."
0,data/frameworks.md,"# Neural Network Frameworks\n\nAs we have learned already, to be able to train neural networks efficiently we need to do two things:\n\n* To operate on tensors, eg. to multiply, add, and compute some functions such as sigmoid or softmax\n* To compute gradients of all expressions, in order to per...","descent optimization While the `numpy` library can do the first part, we need some mechanism to compute gradients. In our framework that we have developed in the previous section we had to manually program all derivative functions inside the `backward` method, which does backpropagation. Ideally..."
0,data/frameworks.md,"# Neural Network Frameworks\n\nAs we have learned already, to be able to train neural networks efficiently we need to do two things:\n\n* To operate on tensors, eg. to multiply, add, and compute some functions such as sigmoid or softmax\n* To compute gradients of all expressions, in order to per...","should give us the opportunity to compute gradients of *any expression* that we can define. Another important thing is to be able to perform computations on GPU, or any other specialized compute units, such as TPU. Deep neural network training requires *a lot* of computations, and to be able to ..."
0,data/frameworks.md,"# Neural Network Frameworks\n\nAs we have learned already, to be able to train neural networks efficiently we need to do two things:\n\n* To operate on tensors, eg. to multiply, add, and compute some functions such as sigmoid or softmax\n* To compute gradients of all expressions, in order to per...","those computations on GPUs is very important. > ✅ The term 'parallelize' means to distribute the computations over multiple devices. Currently, the two most popular neural frameworks are: TensorFlow and PyTorch. Both provide a low-level API to operate with tensors on both CPU and GPU. On top of ..."
0,data/frameworks.md,"# Neural Network Frameworks\n\nAs we have learned already, to be able to train neural networks efficiently we need to do two things:\n\n* To operate on tensors, eg. to multiply, add, and compute some functions such as sigmoid or softmax\n* To compute gradients of all expressions, in order to per...","API, there is also higher-level API, called Keras and PyTorch Lightning correspondingly. Low-Level API | TensorFlow| PyTorch --------------|-------------------------------------|-------------------------------- High-level API| Keras| Pytorch **Low-level APIs** in both frameworks allow you to bui..."


## Converting our text to embeddings

Converting out text  to embeddings, and storing them in our database in chunks

In [37]:
openai.api_type = "azure"
openai.api_key = os.getenv("DAMAI") 
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT") 
openai.api_version = "2023-05-15" # for embeddings



In [51]:
from openai import AzureOpenAI
endpoint= "https://damailearning.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-05-15"
clientEmbeddings = AzureOpenAI(
    api_key=os.getenv('DAMAI'),  # Use the variable instead of accessing os.environ again
    azure_endpoint=endpoint,
    api_version = "2023-05-15",
  )

In [52]:
def create_embeddings(text, model="text-embedding-ada-002"):
    print(text)
    # Create embeddings for each document chunk
    #embeddings = openai.embeddings.create(input = text, model=model).data[0].embedding
    embeddings = clientEmbeddings.embeddings.create(input = [text], model=model).data[0].embedding
    print(embeddings)
    return(embeddings)

#embeddings for the first chunk
create_embeddings(flattened_df['chunks'].iloc[0])

# Neural Network Frameworks As we have learned already, to be able to train neural networks efficiently we need to do two things: * To operate on tensors, eg. to multiply, add, and compute some functions such as sigmoid or softmax * To compute gradients of all expressions, in order to perform gradient
[-0.017071647569537163, 0.002827746095135808, 0.025336062535643578, -0.038865819573402405, 0.006856478750705719, 0.003945613279938698, -0.006249200087040663, -0.0032145045697689056, -0.0029091688338667154, -0.029339350759983063, 0.03490324318408966, 0.020450694486498833, 0.001463914173655212, 0.0030346957501024008, -0.01465610507875681, -0.011019219644367695, 0.02207915112376213, 0.009105783887207508, -0.029339350759983063, -0.020572828128933907, -0.03555462509393692, -0.003704737639054656, 0.01300729252398014, -0.03441470488905907, -0.03042498789727688, -0.0015835039084777236, 0.01544319186359644, -0.043642621487379074, -0.007538394536823034, -0.014235420152544975, 0.01971788890659809, 0

[-0.017071647569537163,
 0.002827746095135808,
 0.025336062535643578,
 -0.038865819573402405,
 0.006856478750705719,
 0.003945613279938698,
 -0.006249200087040663,
 -0.0032145045697689056,
 -0.0029091688338667154,
 -0.029339350759983063,
 0.03490324318408966,
 0.020450694486498833,
 0.001463914173655212,
 0.0030346957501024008,
 -0.01465610507875681,
 -0.011019219644367695,
 0.02207915112376213,
 0.009105783887207508,
 -0.029339350759983063,
 -0.020572828128933907,
 -0.03555462509393692,
 -0.003704737639054656,
 0.01300729252398014,
 -0.03441470488905907,
 -0.03042498789727688,
 -0.0015835039084777236,
 0.01544319186359644,
 -0.043642621487379074,
 -0.007538394536823034,
 -0.014235420152544975,
 0.01971788890659809,
 0.012606964446604252,
 -0.012668031267821789,
 -0.015592467039823532,
 -0.004719130229204893,
 0.011114212684333324,
 0.001191656687296927,
 0.008203347213566303,
 -0.00034668302396312356,
 -0.0017929980531334877,
 0.04033143073320389,
 0.011290628463029861,
 -0.0097910920

In [53]:
cat = create_embeddings("cat")
cat

cat
[-0.007107354234904051, -0.01739795319736004, -0.009769510477781296, -0.03062720224261284, -0.012626631185412407, 0.0031548854894936085, -0.0050478167831897736, -0.04123328998684883, -0.014590458944439888, -0.021311428397893906, 0.019170360639691353, 0.05081846937537193, -0.001183082116767764, 0.002591259777545929, -0.0383690781891346, -0.006128985434770584, 0.03544815257191658, -0.004604714922606945, 0.002406929386779666, -0.013463207520544529, -0.018957670778036118, 0.00901092030107975, 0.01583823375403881, -0.008734424598515034, -0.014562101103365421, 0.007082540541887283, 0.013122905977070332, -0.013243429362773895, 0.0029439690988510847, 0.00485285185277462, 0.00407653721049428, -0.01673152670264244, -0.015753159299492836, -0.04304823651909828, -0.027110746130347252, -0.004317584913223982, 0.008011282421648502, -0.009854585863649845, 0.02206292934715748, -0.009060547687113285, 0.004930838011205196, 0.00023971812333911657, -0.012080729939043522, 0.013158353976905346, -0.0037185

[-0.007107354234904051,
 -0.01739795319736004,
 -0.009769510477781296,
 -0.03062720224261284,
 -0.012626631185412407,
 0.0031548854894936085,
 -0.0050478167831897736,
 -0.04123328998684883,
 -0.014590458944439888,
 -0.021311428397893906,
 0.019170360639691353,
 0.05081846937537193,
 -0.001183082116767764,
 0.002591259777545929,
 -0.0383690781891346,
 -0.006128985434770584,
 0.03544815257191658,
 -0.004604714922606945,
 0.002406929386779666,
 -0.013463207520544529,
 -0.018957670778036118,
 0.00901092030107975,
 0.01583823375403881,
 -0.008734424598515034,
 -0.014562101103365421,
 0.007082540541887283,
 0.013122905977070332,
 -0.013243429362773895,
 0.0029439690988510847,
 0.00485285185277462,
 0.00407653721049428,
 -0.01673152670264244,
 -0.015753159299492836,
 -0.04304823651909828,
 -0.027110746130347252,
 -0.004317584913223982,
 0.008011282421648502,
 -0.009854585863649845,
 0.02206292934715748,
 -0.009060547687113285,
 0.004930838011205196,
 0.00023971812333911657,
 -0.01208072993904

In [54]:
# create embeddings for the whole data chunks and store them in a list

embeddings = []
for chunk in flattened_df['chunks']:
    chunk_str = str(chunk)
    embeddings.append(create_embeddings(chunk_str))

# store the embeddings in the dataframe
flattened_df['embeddings'] = embeddings

flattened_df.head()

# Neural Network Frameworks As we have learned already, to be able to train neural networks efficiently we need to do two things: * To operate on tensors, eg. to multiply, add, and compute some functions such as sigmoid or softmax * To compute gradients of all expressions, in order to perform gradient
[-0.017071647569537163, 0.002827746095135808, 0.025336062535643578, -0.038865819573402405, 0.006856478750705719, 0.003945613279938698, -0.006249200087040663, -0.0032145045697689056, -0.0029091688338667154, -0.029339350759983063, 0.03490324318408966, 0.020450694486498833, 0.001463914173655212, 0.0030346957501024008, -0.01465610507875681, -0.011019219644367695, 0.02207915112376213, 0.009105783887207508, -0.029339350759983063, -0.020572828128933907, -0.03555462509393692, -0.003704737639054656, 0.01300729252398014, -0.03441470488905907, -0.03042498789727688, -0.0015835039084777236, 0.01544319186359644, -0.043642621487379074, -0.007538394536823034, -0.014235420152544975, 0.01971788890659809, 0

,path,text,...,indices,distances
0,data/frameworks.md,"# Neural Network Frameworks\n\nAs we have learned already, to be able to train neural networks efficiently we need to do two things:\n\n* To operate on tensors, eg. to multiply, add, and compute some functions such as sigmoid or softmax\n* To compute gradients of all expressions, in order to per...",...,"[0, 2, 11, 3, 1]","[0.0, 0.5233715652770051, 0.5282499261628433, 0.5461794498898873, 0.5701155822856672]"
0,data/frameworks.md,"# Neural Network Frameworks\n\nAs we have learned already, to be able to train neural networks efficiently we need to do two things:\n\n* To operate on tensors, eg. to multiply, add, and compute some functions such as sigmoid or softmax\n* To compute gradients of all expressions, in order to per...",...,"[1, 0, 32, 2, 50]","[0.0, 0.5701155822856672, 0.592189339823302, 0.6018623661556098, 0.6087011656879424]"
0,data/frameworks.md,"# Neural Network Frameworks\n\nAs we have learned already, to be able to train neural networks efficiently we need to do two things:\n\n* To operate on tensors, eg. to multiply, add, and compute some functions such as sigmoid or softmax\n* To compute gradients of all expressions, in order to per...",...,"[2, 3, 0, 5, 1]","[0.0, 0.5057228769731683, 0.5233715652770051, 0.5707474861697587, 0.6018623661556098]"
0,data/frameworks.md,"# Neural Network Frameworks\n\nAs we have learned already, to be able to train neural networks efficiently we need to do two things:\n\n* To operate on tensors, eg. to multiply, add, and compute some functions such as sigmoid or softmax\n* To compute gradients of all expressions, in order to per...",...,"[3, 2, 0, 10, 11]","[0.0, 0.5057228769731683, 0.5461794498898873, 0.5785683758953769, 0.5867561298437134]"
0,data/frameworks.md,"# Neural Network Frameworks\n\nAs we have learned already, to be able to train neural networks efficiently we need to do two things:\n\n* To operate on tensors, eg. to multiply, add, and compute some functions such as sigmoid or softmax\n* To compute gradients of all expressions, in order to per...",...,"[4, 12, 10, 8, 9]","[0.0, 0.5201643386566065, 0.5530482770720577, 0.5630940019892263, 0.563564546646714]"


Need to push datafram to the Cosmos DB

In [81]:
from azure.cosmos import CosmosClient, PartitionKey
import azure.cosmos
import uuid

# Initialize Cosmos Client
url = os.getenv('COSMOS_DB_ENDPOINT')
key = os.getenv('COSMOS_KEY')
print(url,key[:10])
clientCosmos = CosmosClient(url, credential=key)

# Select database
database_name = 'classdata'
database = clientCosmos.get_database_client(database_name)

# Select container
container_name = 'embeddings'
container = database.get_container_client(container_name)
# Create a new container within the existing database
#container_name = 'embeddingsLesson15'
#container = database.create_container_if_not_exists(
#    id=container_name,
#    partition_key=PartitionKey(path='/id'),
#    offer_throughput=40
#)


for index, row in flattened_df.iterrows():
    item = {
         'id': str(uuid.uuid4()),  # Generate a unique ID
        'path': row['path'],
        'text': row['text'],
        'chunks': row['chunks'],
        'embeddings': row['embeddings']
    }
    try:
        container.create_item(body=item)
    except Exception as e:
        print(f"Error: {str(e)}")

print("Data has been successfully pushed to the Cosmos DB container.")

https://damcosmos.documents.azure.com:443/ p3vG5uYg7G
Data has been successfully pushed to the Cosmos DB container.


# Retrieval

Vector search and similiarity between our prompt and the database

### Creating an search index and reranking

In [86]:
from sklearn.neighbors import NearestNeighbors

query = "SELECT c.id, c.path, c.text, c.chunks, c.embeddings FROM c"
items = container.query_items(
    query=query,
    enable_cross_partition_query=True
)

# Convert items to DataFrame
data = []
for item in items:
    data.append(item)
cosmosdb_df = pd.DataFrame(data)
embeddings = cosmosdb_df['embeddings'].to_list()
##embeddings = flattened_df['embeddings'].to_list()

# Create the search index
nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(embeddings)

# To query the index, you can use the kneighbors method
distances, indices = nbrs.kneighbors(embeddings)

# Store the indices and distances in the DataFrame
##flattened_df['indices'] = indices.tolist()
##flattened_df['distances'] = distances.tolist()
##
##flattened_df.head()

cosmosdb_df['indices2'] = indices.tolist()
cosmosdb_df['distances2'] = distances.tolist()

cosmosdb_df.head()

,id,path,...,indices2,distances2
0,0,data/frameworks.md,...,"[3, 0, 5, 14, 6]","[0.0, 0.0, 0.5233715652770051, 0.5282499261628433, 0.5461794498898873]"
1,1,data/own_framework.md,...,"[1, 28, 46, 2, 44]","[0.0, 0.0, 0.45846243712732504, 0.45846243712732504, 0.5057848433822951]"
2,2,data/perceptron.md,...,"[46, 2, 28, 1, 49]","[0.0, 0.0, 0.45846243712732504, 0.45846243712732504, 0.5236378295599984]"
3,4e88f1c5-408f-4c74-af08-c8386085eff9,data/frameworks.md,...,"[3, 0, 5, 14, 6]","[0.0, 0.0, 0.5233715652770051, 0.5282499261628433, 0.5461794498898873]"
4,55bd6271-f689-4443-8502-7975cce4de20,data/frameworks.md,...,"[4, 3, 0, 35, 5]","[0.0, 0.5701155822856672, 0.5701155822856672, 0.592189339823302, 0.6018623661556098]"


In [56]:
# Your text question
question = "what is a perceptron?"

# Convert the question to a query vector
query_vector = create_embeddings(question)  # You need to define this function

# Find the most similar documents
distances, indices = nbrs.kneighbors([query_vector])

index = []
# Print the most similar documents
for i in range(3):
    index = indices[0][i]
    for index in indices[0]:
        print(flattened_df['chunks'].iloc[index])
        print(flattened_df['path'].iloc[index])
        print(flattened_df['distances'].iloc[index])
    else:
        print(f"Index {index} not found in DataFrame")

what is a perceptron?
[-0.03205700218677521, 0.0044459146447479725, -0.010059516876935959, -0.014266091398894787, -0.00461997976526618, 0.012264342047274113, 0.010501932352781296, -0.008558204397559166, -0.007579087745398283, -0.03446490317583084, 0.017275968566536903, 0.05346702039241791, -0.028648225590586662, -0.006610850337892771, 0.013519061729311943, 0.010646986775100231, 0.022846054285764694, -0.0008793917368166149, -0.0035085012204945087, 0.002152243396267295, -0.012423901818692684, -0.019973978400230408, -0.017333989962935448, -0.037046872079372406, -0.0028285591397434473, -0.009479299187660217, 0.029533058404922485, -0.027850428596138954, -0.005396020598709583, -0.02210627682507038, 0.0255440641194582, -0.02056870050728321, 0.007898206822574139, -0.01624608226120472, 0.004420530050992966, -0.0025837798602879047, 0.022613966837525368, -0.012380385771393776, 0.019756397232413292, 0.02290407568216324, 0.02194671705365181, 0.01568037085235119, -0.012764778919517994, -0.0063497526

## Putting it all together to answer a question

In [58]:
import os
from openai import AzureOpenAI

# Access the API key
api_key = os.getenv('DAMAI')
deployment = os.environ['AZURE_OPENAI_DEPLOYMENT']
endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')

print(api_key[:10])  # This should print your API key
print(deployment)  # This should print your deployment key
print(endpoint)  # This should print your deployment key


# Initialize the AzureOpenAI client
clientAOI = AzureOpenAI(
    api_key=api_key,  # Use the variable instead of accessing os.environ again
    azure_endpoint=endpoint,
    api_version = "2024-08-01-preview"
  )

XwU7KK9sui
gpt4mdam
https://damailearning.openai.azure.com/openai/deployments/gpt4mdam/chat/completions?api-version=2024-08-01-preview


In [59]:
user_input = "what is a perceptron?"

def chatbot(user_input):
    # Convert the question to a query vector
    query_vector = create_embeddings(user_input)

    # Find the most similar documents
    distances, indices = nbrs.kneighbors([query_vector])

    # add documents to query  to provide context
    history = []
    for index in indices[0]:
        history.append(flattened_df['chunks'].iloc[index])

    # combine the history and the user input
    history.append(user_input)

    # create a message object
    messages=[
        {"role": "system", "content": "You are an AI assiatant that helps with AI questions."},
        {"role": "user", "content": history[-1]}
    ]

    # use chat completion to generate a response
    response = clientAOI.chat.completions.create(
        model=deployment,
        temperature=0.7,
        max_tokens=800,
        messages=messages
    )

    return response.choices[0].message

chatbot(user_input)

what is a perceptron?
[-0.03205700218677521, 0.0044459146447479725, -0.010059516876935959, -0.014266091398894787, -0.00461997976526618, 0.012264342047274113, 0.010501932352781296, -0.008558204397559166, -0.007579087745398283, -0.03446490317583084, 0.017275968566536903, 0.05346702039241791, -0.028648225590586662, -0.006610850337892771, 0.013519061729311943, 0.010646986775100231, 0.022846054285764694, -0.0008793917368166149, -0.0035085012204945087, 0.002152243396267295, -0.012423901818692684, -0.019973978400230408, -0.017333989962935448, -0.037046872079372406, -0.0028285591397434473, -0.009479299187660217, 0.029533058404922485, -0.027850428596138954, -0.005396020598709583, -0.02210627682507038, 0.0255440641194582, -0.02056870050728321, 0.007898206822574139, -0.01624608226120472, 0.004420530050992966, -0.0025837798602879047, 0.022613966837525368, -0.012380385771393776, 0.019756397232413292, 0.02290407568216324, 0.02194671705365181, 0.01568037085235119, -0.012764778919517994, -0.0063497526

ChatCompletionMessage(content='A perceptron is a type of artificial neural network and one of the simplest forms of a neural network model. It was introduced by Frank Rosenblatt in the 1950s and is a fundamental building block for understanding more complex neural networks.\n\n### Key Characteristics of a Perceptron:\n\n1. **Structure**: A perceptron consists of one or more input nodes (features), a single output node, and weights associated with each input. It can be visualized as a weighted sum of inputs passed through an activation function.\n\n2. **Activation Function**: The perceptron uses a step function (or threshold function) as its activation function. The output of the perceptron is typically binary: it produces an output of 1 (or "true") if the weighted sum of the inputs exceeds a certain threshold, and 0 (or "false") otherwise.\n\n3. **Learning**: The perceptron learns by adjusting its weights based on the errors it makes during training. This is typically done using a meth

## Testing and evaluation

A basic example of how you can use Mean Average Precision (MAP) to evaluate the responses of your model based on their relevance.

In [60]:
from sklearn.metrics import average_precision_score

# Define your test cases
test_cases = [
    {
        "query": "What is a perceptron?",
        "relevant_responses": ["A perceptron is a type of artificial neuron.", "It's a binary classifier used in machine learning."],
        "irrelevant_responses": ["A perceptron is a type of fruit.", "It's a type of car."]
    },
    {
        "query": "What is machine learning?",
        "relevant_responses": ["Machine learning is a method of data analysis that automates analytical model building.", "It's a branch of artificial intelligence based on the idea that systems can learn from data, identify patterns and make decisions with minimal human intervention."],
        "irrelevant_responses": ["Machine learning is a type of fruit.", "It's a type of car."]
    },
    {
        "query": "What is deep learning?",
        "relevant_responses": ["Deep learning is a subset of machine learning in artificial intelligence (AI) that has networks capable of learning unsupervised from data that is unstructured or unlabeled.", "It's a type of machine learning."],
        "irrelevant_responses": ["Deep learning is a type of fruit.", "It's a type of car."]
    },
    {
        "query": "What is a neural network?",
        "relevant_responses": ["A neural network is a series of algorithms that endeavors to recognize underlying relationships in a set of data through a process that mimics the way the human brain operates.", "It's a type of machine learning."],
        "irrelevant_responses": ["A neural network is a type of fruit.", "It's a type of car."]
    }
]

# Initialize the total average precision
total_average_precision = 0

# Test the RAG application
for test_case in test_cases:
    query = test_case["query"]
    relevant_responses = test_case["relevant_responses"]
    irrelevant_responses = test_case["irrelevant_responses"]

    # Generate a response using your RAG application
    response = chatbot(query) 

    # Create a list of all responses and a list of true binary labels
    all_responses = relevant_responses + irrelevant_responses
    true_labels = [1] * len(relevant_responses) + [0] * len(irrelevant_responses)

    # Create a list of predicted scores based on whether the response is the generated response
    predicted_scores = [1 if resp == response else 0 for resp in all_responses]

    # Calculate the average precision for this query
    average_precision = average_precision_score(true_labels, predicted_scores)

    # Add the average precision to the total average precision
    total_average_precision += average_precision

# Calculate the mean average precision
mean_average_precision = total_average_precision / len(test_cases)

What is a perceptron?
[-0.026461370289325714, 0.004558384418487549, -0.0024790307506918907, -0.007938411086797714, -0.004348209593445063, 0.010019487701356411, 0.012107455171644688, -0.0012386540183797479, -0.001896742731332779, -0.034647855907678604, 0.020397305488586426, 0.0501939095556736, -0.02724694274365902, -0.008317414671182632, 0.012465786188840866, 0.009399299509823322, 0.026282204315066338, -0.003531628055498004, -0.00463418522849679, 0.005402529612183571, -0.02116909623146057, -0.018784815445542336, -0.019267184659838676, -0.02943827398121357, -0.0019570388831198215, -0.006856526713818312, 0.032470304518938065, -0.018633214756846428, 0.00815892219543457, -0.02229921706020832, 0.02839084528386593, -0.0223405621945858, 0.0034678864758461714, -0.012555369175970554, 0.001292059081606567, 0.002975181443616748, 0.022754020988941193, -0.006808289792388678, 0.018522959202528, 0.015201506204903126, 0.024683495983481407, 0.025152083486318588, -0.015504708513617516, -0.010591438971459

In [61]:
mean_average_precision

np.float64(0.5)

THIS IS  PUTTING IT ALL INTO ACTIONS

In [91]:
##Fix the DB so it has the embeddings 
# Get embeddings from cosmos db
# Prompt it
# create embeddings for prompt
# find the nearest neightbors
# get that data
# Add chunks of data to query

import os
from openai import AzureOpenAI
from azure.cosmos import CosmosClient, PartitionKey
import azure.cosmos
import uuid
from sklearn.neighbors import NearestNeighbors

# Initialize Cosmos Client
url = os.getenv('COSMOS_DB_ENDPOINT')
key = os.getenv('COSMOS_KEY')
clientCosmos = CosmosClient(url, credential=key)

def GetDataCosmos():
    query = "SELECT c.id, c.path, c.text, c.chunks, c.embeddings FROM c"
    items = container.query_items(
        query=query,
        enable_cross_partition_query=True
    )
    # Convert items to DataFrame
    data = []
    for item in items:
        data.append(item)
    cosmosdb_df = pd.DataFrame(data)
    embeddings = cosmosdb_df['embeddings'].to_list()
    # Create the search index
    nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(embeddings)

    # To query the index, you can use the kneighbors method
    distances, indices = nbrs.kneighbors(embeddings)

    # Store the indices and distances in the DataFrame
    cosmosdb_df['indices2'] = indices.tolist()
    cosmosdb_df['distances2'] = distances.tolist()

    cosmosdb_df.head()
    return cosmosdb_df

def StoreDataCosmos():
    # Select database
    database_name = 'classdata'
    database = clientCosmos.get_database_client(database_name)

    # Select container
    container_name = 'embeddings'
    container = database.get_container_client(container_name)

    for index, row in flattened_df.iterrows():
        item = {
            'id': str(uuid.uuid4()),  # Generate a unique ID
            'path': row['path'],
            'text': row['text'],
            'chunks': row['chunks'],
            'embeddings': row['embeddings']
        }
        try:
            container.create_item(body=item)
        except Exception as e:
            print(f"Error: {str(e)}")

    print("Data has been successfully pushed to the Cosmos DB container.")

def create_embeddings(text, model="text-embedding-ada-002"):
    endpoint= "https://damailearning.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-05-15"
    clientEmbeddings = AzureOpenAI(
     api_key=os.getenv('DAMAI'),  # Use the variable instead of accessing os.environ again
     azure_endpoint=endpoint,
     api_version = "2023-05-15",
    )
    print(text)
    # Create embeddings for each document chunk
    #embeddings = openai.embeddings.create(input = text, model=model).data[0].embedding
    embeddings = clientEmbeddings.embeddings.create(input = [text], model=model).data[0].embedding
    print(embeddings)
    return(embeddings)

def chatbot(user_input):
    # Access the API key
    api_key = os.getenv('DAMAI')
    deployment = os.environ['AZURE_OPENAI_DEPLOYMENT']
    endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
    # Initialize the AzureOpenAI client
    clientAOI = AzureOpenAI(
        api_key=api_key,  # Use the variable instead of accessing os.environ again
        azure_endpoint=endpoint,
        api_version = "2024-08-01-preview"
    )

    # Convert the question to a query vector
    query_vector = create_embeddings(user_input)

    # Find the most similar documents
    distances, indices = nbrs.kneighbors([query_vector])

    # add documents to query  to provide context
    flattened_df = GetDataCosmos()
    history = []
    for index in indices[0]:
        history.append(flattened_df['chunks'].iloc[index])

    # combine the history and the user input
    history.append(user_input)

    # create a message object
    messages=[
        {"role": "system", "content": "You are an AI assiatant that helps with AI questions.use only the data passed into the history to answer the question.  If you don't know say you don't know"},
        {"role": "user", "content": history[-1]}
    ]

    # use chat completion to generate a response
    response = clientAOI.chat.completions.create(
        model=deployment,
        temperature=0.7,
        max_tokens=800,
        messages=messages
    )

    return response.choices[0].message

user_input = "what is a ai?"
chatbot(user_input)

what is a ai?
[-0.007490857969969511, -0.017605308443307877, -0.026838060468435287, 0.0001541181409265846, -0.0119423633441329, 0.015397476963698864, -0.03288809210062027, 0.001853001769632101, 0.008910179138183594, -0.03174116834998131, 0.029361296445131302, 0.010623399168252945, 0.0015528298681601882, 0.004042912740260363, -0.0031397088896483183, 0.006541060283780098, 0.026164241135120392, 0.010085777379572392, 0.009096554480493069, -0.004852929152548313, -0.040887899696826935, 0.03397767245769501, -0.008573269471526146, -0.017547963187098503, -0.010953140445053577, 0.0013117962516844273, 0.013010437600314617, -0.04478744789958, -0.00761271920055151, -0.02708178199827671, 0.024501198902726173, 0.0007826871587894857, -0.018121425062417984, 6.171445420477539e-05, -0.0005407575517892838, 0.001033129170536995, -0.002236505039036274, 0.0003593103028833866, 0.013598237186670303, 0.016228998079895973, 0.03876608610153198, 0.006175477989017963, -0.00950514618307352, 0.0005989998462609947, -0

ChatCompletionMessage(content='AI, or artificial intelligence, refers to the simulation of human intelligence in machines that are programmed to think and learn. These systems can perform tasks that typically require human intelligence, such as understanding natural language, recognizing patterns, solving problems, and making decisions. AI encompasses various technologies, including machine learning, natural language processing, and robotics.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)